# Midterm Project


#### Load our Python tools

In [2]:
# %load_ext lab_black

In [3]:
import pandas as pd
import altair as alt

In [4]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

---

#### Import statewide dataset

In [5]:
covid_data=pd.read_csv("https://raw.githubusercontent.com/datadesk/california-coronavirus-data/master/cdph-state-cases-deaths.csv").sort_values('date')

#### Data are cumulative, so we need to extract how many new cases there were each day

In [6]:
covid_data['daily_new_cases'] = covid_data['confirmed_cases'].diff().fillna(covid_data['confirmed_cases'])

In [7]:
covid_data.head()

,date,confirmed_cases,reported_cases,probable_cases,reported_and_probable_cases,reported_deaths,daily_new_cases
774,2020-02-01,23,3,1,4,0,23.0
773,2020-02-02,30,6,1,7,0,7.0
772,2020-02-03,35,6,1,7,0,5.0
771,2020-02-04,35,6,2,8,0,0.0
770,2020-02-05,38,6,2,8,0,3.0


In [8]:
covid_data.tail()

,date,confirmed_cases,reported_cases,probable_cases,reported_and_probable_cases,reported_deaths,daily_new_cases
4,2022-03-12,8442881,8426700,594634,9021334,86387,1110.0
3,2022-03-13,8443584,8438328,594726,9033054,86794,703.0
2,2022-03-14,8444691,8439055,594962,9034017,86792,1107.0
1,2022-03-15,8444888,8442537,595018,9037555,86927,197.0
0,2022-03-16,8444888,8445468,595018,9040486,87045,0.0


#### Create a generic month column for aggregation

In [9]:
covid_data["month"]=pd.to_datetime(covid_data["date"]).dt.strftime("%Y-%m-01")

In [10]:
covid_data.head()

,date,confirmed_cases,reported_cases,probable_cases,reported_and_probable_cases,reported_deaths,daily_new_cases,month
774,2020-02-01,23,3,1,4,0,23.0,2020-02-01
773,2020-02-02,30,6,1,7,0,7.0,2020-02-01
772,2020-02-03,35,6,1,7,0,5.0,2020-02-01
771,2020-02-04,35,6,2,8,0,0.0,2020-02-01
770,2020-02-05,38,6,2,8,0,3.0,2020-02-01


#### Group by month to sum new cases by month

In [11]:
covid_group_months = covid_data.groupby(["month"]).agg({"daily_new_cases":"sum"}).reset_index()

#### How many new cases per month? 

In [12]:
covid_group_months.rename(columns={'daily_new_cases':'new_monthly_cases'}, inplace=True)

In [13]:
covid_group_months.head()

,month,new_monthly_cases
0,2020-02-01,466.0
1,2020-03-01,19259.0
2,2020-04-01,46544.0
3,2020-05-01,71156.0
4,2020-06-01,173946.0


In [14]:
covid_group_months["new_monthly_covid_cases_per_100k"]=((covid_group_months.new_monthly_cases/38000000)*100000).round(1)

In [15]:
covid_group_months

,month,new_monthly_cases,new_monthly_covid_cases_per_100k
0,2020-02-01,466.0,1.2
1,2020-03-01,19259.0,50.7
2,2020-04-01,46544.0,122.5
3,2020-05-01,71156.0,187.3
4,2020-06-01,173946.0,457.8
5,2020-07-01,280811.0,739.0
6,2020-08-01,149064.0,392.3
7,2020-09-01,98757.0,259.9
8,2020-10-01,120593.0,317.4
9,2020-11-01,399991.0,1052.6


#### Now that you have Altair!

In [16]:
#### Omicron pops!

In [17]:
alt.Chart(covid_group_months).mark_bar(size=10).encode(
    x=alt.X('month:T', axis=alt.Axis(format="%b. %y")),
    y='new_monthly_cases:Q'
).properties(title='New CA COVID cases by month')

alt.Chart(...)

---

#### Merge with flights

In [18]:
flight_data=pd.read_csv("data/Los_Angeles_International_Airport_-_Flight_Operations_By_Month.csv", dtype = {"DataExtractDate":str})

In [19]:
flight_data["clean_date"]= pd.to_datetime(flight_data["ReportPeriod"]).dt.strftime("%Y-%m-%d")

In [20]:
flight_data.head()

,DataExtractDate,ReportPeriod,FlightType,Arrival_Departure,Domestic_International,FlightOpsCount,clean_date
0,04/01/2021 09:37:48 AM,02/01/2021 12:00:00 AM,Scheduled,Departure,International,1690,2021-02-01
1,04/18/2018 03:12:41 PM,12/01/2017 12:00:00 AM,Scheduled,Departure,Domestic,20797,2017-12-01
2,04/18/2018 03:12:41 PM,10/01/2008 12:00:00 AM,Charter,Departure,International,9,2008-10-01
3,04/18/2018 03:12:41 PM,02/01/2016 12:00:00 AM,Scheduled,Arrival,Domestic,18427,2016-02-01
4,04/18/2018 03:12:41 PM,09/01/2007 12:00:00 AM,Scheduled,Departure,International,4227,2007-09-01


In [21]:
flight_data[flight_data["clean_date"]=="2018-04-17"]

,DataExtractDate,ReportPeriod,FlightType,Arrival_Departure,Domestic_International,FlightOpsCount,clean_date


In [22]:
flight_group =  flight_data.groupby(["clean_date", 'Domestic_International'])["FlightOpsCount"].sum().reset_index()

In [23]:
flight_group_type = flight_group.pivot_table(columns='Domestic_International',  index='clean_date', values='FlightOpsCount').reset_index()

In [24]:
flight_group_type['Total'] = flight_group_type['Domestic'] + flight_group_type['International']

In [25]:
flight_group_type.head()

Domestic_International,clean_date,Domestic,International,Total
0,2006-01-01,39471,8477,47948
1,2006-02-01,35723,7036,42759
2,2006-03-01,40419,7983,48402
3,2006-04-01,38761,7845,46606
4,2006-05-01,40492,8042,48534


In [26]:
covid_flight_merge=pd.merge(covid_group_months, flight_group_type, left_on='month', right_on='clean_date')

In [27]:
covid_flight_merge.head()

,month,new_monthly_cases,new_monthly_covid_cases_per_100k,clean_date,Domestic,International,Total
0,2020-02-01,466.0,1.2,2020-02-01,38039,8287,46326
1,2020-03-01,19259.0,50.7,2020-03-01,35547,6492,42039
2,2020-04-01,46544.0,122.5,2020-04-01,9903,1198,11101
3,2020-05-01,71156.0,187.3,2020-05-01,8240,1120,9360
4,2020-06-01,173946.0,457.8,2020-06-01,11184,1316,12500


#### What month had the highest covid rate per 100k?

In [28]:
covid_flight_merge.sort_values("new_monthly_covid_cases_per_100k", ascending=False)

,month,new_monthly_cases,new_monthly_covid_cases_per_100k,clean_date,Domestic,International,Total
10,2020-12-01,1163615.0,3062.1,2020-12-01,21221,4105,25326
11,2021-01-01,834492.0,2196.0,2021-01-01,20285,4558,24843
22,2021-12-01,714621.0,1880.6,2021-12-01,32070,7038,39108
9,2020-11-01,399991.0,1052.6,2020-11-01,19796,3282,23078
18,2021-08-01,387626.0,1020.1,2021-08-01,35818,6191,42009
5,2020-07-01,280811.0,739.0,2020-07-01,18047,2082,20129
19,2021-09-01,231502.0,609.2,2021-09-01,33353,5559,38912
17,2021-07-01,189647.0,499.1,2021-07-01,35828,5975,41803
4,2020-06-01,173946.0,457.8,2020-06-01,11184,1316,12500
12,2021-02-01,170676.0,449.1,2021-02-01,18102,3310,21412


#### How many unique months have a level 4 travel health notice level? (covid rate more than 500 cases per 100k)

In [72]:
max_covid = covid_flight_merge[covid_flight_merge['new_monthly_covid_cases_per_100k'] >= 500]

In [73]:
len(max_covid)

7

#### How many unique months have a level 1 travel health notice level? (covid rate less than 50 cases per 100k)

In [74]:
min_covid = covid_flight_merge[covid_flight_merge['new_monthly_covid_cases_per_100k'] <= 50]

In [75]:
len(min_covid)

1

#### When and how many covid cases per 100k occured when international flight operations were at its lowest?

In [82]:
covid_flight_merge[covid_flight_merge['International'] == covid_flight_merge["International"].min()]

,month,new_monthly_cases,new_monthly_covid_cases_per_100k,clean_date,Domestic,International,Total
3,2020-05-01,71160.0,187.3,2020-05-01,8240,1120,9360


#### When and how many covid cases per 100k occured when domestic flight operations were at its highest?

In [83]:
covid_flight_merge[covid_flight_merge['Domestic'] == covid_flight_merge["Domestic"].max()]

,month,new_monthly_cases,new_monthly_covid_cases_per_100k,clean_date,Domestic,International,Total
0,2020-02-01,466.0,1.2,2020-02-01,38039,8287,46326


#### When and how many flights occured when covid rate was at its lowest?

In [41]:
covid_flight_merge[covid_flight_merge['new_monthly_covid_cases_per_100k'] == covid_flight_merge["new_monthly_covid_cases_per_100k"].min()]

,month,new_monthly_cases,new_monthly_covid_cases_per_100k,clean_date,Domestic,International,Total
0,2020-02-01,466.0,1.2,2020-02-01,38039,8287,46326


#### What's the average covid rate for each international flight of the month?

In [63]:
flight_ops_international_data=covid_flight_merge.groupby(["International"])["new_monthly_covid_cases_per_100k"].mean().reset_index()

In [64]:
flight_ops_international_data

,International,new_monthly_covid_cases_per_100k
0,1120,187.3
1,1198,122.5
2,1316,457.8
3,2082,739.0
4,2376,392.3
5,2521,259.9
6,3031,317.3
7,3282,1052.6
8,3310,449.1
9,3448,201.8
